![logo](IBM-logo.jpg)

![logo](coursera.png)

# IBM Capstone Data Science Project

**Table of contents**
1. [Data acquisition](#Chapter1) 
2. [Data cleaning](#Chapter2)
3. [Analysis](Chapter3)

This notebook will be used to demonstrate the learned skills within the IBM data science course on Coursera.

In [1]:
import pandas as pd # Library for dataframe manipulations
import numpy as np # library for mathematical functions
import requests # library to handle requests
import random # library for random number generation
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values
import folium # plotting library
import geocoder # import geocoder

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 

# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize
import json
import csv
from json import load

print('Libraries imported.')

Libraries imported.


## 1. Data acquisition <a name="Chapter1"></a>

To aquire the data for the clustering of Toronto webscraping is used. To be able to webscrape the data from [Wikipedia](https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M), I use the package [pandas](https://pandas.pydata.org/pandas-docs/stable/).

In [2]:
tables = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M', header=0) # to acces the wikipedia page

# now to construct the dataframe using pandas
headings = ['Postcode', 'Borough', 'Neighbourhood']
for table in tables:
    current_headings = table.columns.values[:3]
    if list(current_headings) == headings:
        break


In [3]:
table['Postcode'].replace({r'.*!(.*)': r'\1'}, regex=True, inplace=True)
table['Borough'].replace({r'.*!(.*)': r'\1'}, regex=True, inplace=True)
table['Neighbourhood'].replace({r'.*!(.*)': r'\1'}, regex=True, inplace=True)

table[headings].to_csv('test.txt', sep=',', header=False, index=False)

In [4]:
df = pd.DataFrame(table)
df.head()

Postcode           Borough     Neighbourhood
0      M1A      Not assigned      Not assigned
1      M2A      Not assigned      Not assigned
2      M3A        North York         Parkwoods
3      M4A        North York  Victoria Village
4      M5A  Downtown Toronto      Harbourfront

## 2. Data cleaning <a name="Chapter2"></a>

At this stage the dataset will be assesed. If there are any missing values or other values that could impact the analysis they are removed or dealt with accordingly.

In [5]:
# Adressing the not assigned values
df['Borough'].replace('Not assigned', np.NaN, inplace=True)
df.dropna(inplace=True)
# to verify that it worked
df.head()

Postcode           Borough     Neighbourhood
2      M3A        North York         Parkwoods
3      M4A        North York  Victoria Village
4      M5A  Downtown Toronto      Harbourfront
5      M6A        North York  Lawrence Heights
6      M6A        North York    Lawrence Manor

In [6]:
# grouping values based on zipcode
df = df.groupby(['Postcode', 'Borough'])['Neighbourhood'].apply(', '.join).reset_index()

In [7]:
# shape of the current dataframe
print(df.shape)

(103, 3)


###  Geospatial Coordinates 

In [8]:
df2 = pd.read_csv('Geospatial_Coordinates.csv')
df2 = pd.DataFrame(df2)

In [9]:
df2.rename(columns = {'Postal Code': 'Postcode'}, inplace=True)

In [10]:
df_new = pd.merge(df, df2, on='Postcode')

In [11]:
df_new

Postcode      Borough                                      Neighbourhood  \
0        M1B  Scarborough                                     Rouge, Malvern   
1        M1C  Scarborough             Highland Creek, Rouge Hill, Port Union   
2        M1E  Scarborough                  Guildwood, Morningside, West Hill   
3        M1G  Scarborough                                             Woburn   
4        M1H  Scarborough                                          Cedarbrae   
..       ...          ...                                                ...   
98       M9N         York                                             Weston   
99       M9P    Etobicoke                                          Westmount   
100      M9R    Etobicoke  Kingsview Village, Martin Grove Gardens, Richv...   
101      M9V    Etobicoke  Albion Gardens, Beaumond Heights, Humbergate, ...   
102      M9W    Etobicoke                                          Northwest   

      Latitude  Longitude  
0    43.806686 -79.194353  
1    43.784535 -79.160497  
2    43.763573 -79.188711  
3    43.770992 -79.216917  
4    43.773136 -79.239476  
..         ...        ...  
98   43.706876 -79.518188  
99   43.696319 -79.532242  
100  43.688905 -79.554724  
101  43.739416 -79.588437  
102  43.706748 -79.594054  

[103 rows x 5 columns]

## 3. Analysis <a name="Chapter3"></a>

At this stage the different neighbourhoods will be clustered and visualized.

In [12]:
address = 'Toronto'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(f'The geograpical coordinate of Toronto are {latitude}, {longitude}.')

The geograpical coordinate of Toronto are 43.653963, -79.387207.


### Creating a map from Toronto

The first step is to generate a map of Toronto and attach markers using the created dataframe from before.

In [14]:
# to generate the latitude and longitude of Toronto.
locations = df_new[['Latitude', 'Longitude']]
locations_list = locations.values.tolist()

In [38]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude],zoom_start=11)

# Ger a nice layer over the folium map
folium.TileLayer('cartodbdark_matter').add_to(map_toronto)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_new['Latitude'], df_new['Longitude'], df_new['Borough'], df_new['Neighbourhood']):
    label = f'{neighborhood}, {borough}'
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)  
    
# to get the output    
map_toronto